## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-11-11-10-14 +0000,bbc,What we know about the abduction of Savannah G...,https://www.bbc.com/news/articles/cdex1nlrj07o...
1,2026-02-11-11-09-51 +0000,nypost,Ex-Prince Andrew’s gushing emails to Epstein b...,https://nypost.com/2026/02/11/world-news/ex-pr...
2,2026-02-11-11-02-25 +0000,bbc,Netanyahu to meet Trump as Iran nuclear talks ...,https://www.bbc.com/news/articles/c0k1j2j4607o...
3,2026-02-11-11-02-00 +0000,wsj,Israeli President’s Visit to Australia Splits ...,https://www.wsj.com/world/israeli-presidents-v...
4,2026-02-11-11-01-45 +0000,bbc,Lib Dems plan to replace Treasury with growth ...,https://www.bbc.com/news/articles/cn71mlmjn5mo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2422/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
22,trump,50
9,epstein,23
2,guthrie,16
146,canada,15
197,new,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
107,2026-02-11-03-37-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...,159
241,2026-02-10-20-53-18 +0000,nyt,Former Palm Beach Police Chief Said Trump Told...,https://www.nytimes.com/2026/02/10/us/politics...,107
104,2026-02-11-03-51-40 +0000,bbc,Under-fire Trump commerce secretary confirms h...,https://www.bbc.com/news/articles/cwyx9vq0ne0o...,105
244,2026-02-10-20-48-27 +0000,bbc,Ex-police chief said Trump told him in 2006 'e...,https://www.bbc.com/news/articles/cx2407jrn83o...,102
218,2026-02-10-22-09-00 +0000,wsj,Epstein Files Appear to Show Effort to Get Tru...,https://www.wsj.com/politics/policy/epstein-fi...,96


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
107,159,2026-02-11-03-37-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...
121,76,2026-02-11-02-45-29 +0000,nyt,New Photos Show Masked Person on Nancy Guthrie...,https://www.nytimes.com/2026/02/10/us/nancy-gu...
1,59,2026-02-11-11-09-51 +0000,nypost,Ex-Prince Andrew’s gushing emails to Epstein b...,https://nypost.com/2026/02/11/world-news/ex-pr...
60,52,2026-02-11-09-03-03 +0000,nypost,Tumbler Ridge students barricaded themselves i...,https://nypost.com/2026/02/11/world-news/tumbl...
342,46,2026-02-10-16-07-54 +0000,nypost,Nervous Netanyahu pushes up trip to DC for hig...,https://nypost.com/2026/02/10/us-news/nervous-...
339,45,2026-02-10-16-15-01 +0000,nypost,Gavin Newsom ripped by LA’s top fed prosecuto...,https://nypost.com/2026/02/10/us-news/gavin-ne...
188,41,2026-02-10-23-03-05 +0000,latimes,$600 million in Trump administration health cu...,https://www.latimes.com/politics/story/2026-02...
158,36,2026-02-11-00-21-00 +0000,wsj,Estée Lauder Files Lawsuit Against Walmart All...,https://www.wsj.com/business/retail/estee-laud...
237,35,2026-02-10-21-15-41 +0000,nypost,LI parents who tragically lost hockey star son...,https://nypost.com/2026/02/10/us-news/li-paren...
161,35,2026-02-11-00-12-21 +0000,latimes,LAPD to train their body cameras on immigratio...,https://www.latimes.com/california/story/2026-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
